In [1]:
import os
import re
import json
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime


In [2]:
df = pd.read_csv("Files/apartmentsPortugal.csv")
df

,Unnamed: 0,Title,Price,Location,Rooms,Area,URL,DateScraped,Useful area,Gross area,Construction year,Energetic certificate,Enterprise,Description,useful_area,gross_area,construction_year,energetic_certificate,enterprise,description
0,0,T3 Duplex | Centro Maia | 180 m2,299 000 €,"Rua 1º de Maio - Gueifães, Centro, Cidade da M...",T3,160.00,https://www.imovirtual.com/pt/anuncio/t3-duple...,2025-01-21,NaN,160m²,Aquecimento:,Aquecimento:,NaN,NaN,NaN,160m²,Aquecimento:,Aquecimento:,NaN,NaN
1,1,Excelente Apartamento T2 em Monte do Estoril -...,380 000 €,"Praceta Marquesa do Cadaval, Monte Estoril, Ca...",T2,66.00,https://www.imovirtual.com/pt/anuncio/excelent...,2025-01-21,NaN,66m²,Aquecimento:,Aquecimento:,NaN,NaN,NaN,66m²,Aquecimento:,Aquecimento:,NaN,NaN
2,2,Apartamento T4 +1 as Lezírias do Infantado,Preço sob consulta,"Alameda Dom João II - Infantado, Infantado, Lo...",T4,182.00,https://www.imovirtual.com/pt/anuncio/apartame...,2025-01-21,NaN,182m²,Aquecimento:,Aquecimento:,NaN,NaN,NaN,182m²,Aquecimento:,Aquecimento:,NaN,NaN
3,3,T4 Boavista - foco de luxo - Terraço,830 000 €,"Bessa - Foco, Ramalde, Porto, Porto",T4,172.65,https://www.imovirtual.com/pt/anuncio/t4-boavi...,2025-01-21,NaN,172.65m²,Aquecimento:,Aquecimento:,NaN,NaN,NaN,172.65m²,Aquecimento:,Aquecimento:,NaN,NaN
4,4,"Magnifico T3, varanda e box, junto ao Parque d...",455 000 €,"Prelada - Monte dos Burgos, Ramalde, Porto, Porto",T3,137.00,https://www.imovirtual.com/pt/anuncio/magnific...,2025-01-21,NaN,137m²,Aquecimento:,Aquecimento:,NaN,NaN,NaN,137m²,Aquecimento:,Aquecimento:,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,75,Apartamento T3 Remodelado em Setúbal,265 000 €,"Avenida Jaime Cortesão, Arangues - Pinheirinho...",T3,105.00,https://www.imovirtual.com/pt/anuncio/apartame...,2025-01-21,NaN,105m²,Aquecimento:,Aquecimento:,NaN,NaN,NaN,105m²,Aquecimento:,Aquecimento:,NaN,NaN
76,76,"Apartamento T3 | Costa, Guimarães",265 000 €,"Rua Eça de Queirós - Costa, Costa, Guimarães, ...",T3,110.00,https://www.imovirtual.com/pt/anuncio/apartame...,2025-01-21,NaN,110m²,Aquecimento:,Aquecimento:,NaN,NaN,NaN,110m²,Aquecimento:,Aquecimento:,NaN,NaN
77,77,Espaçoso apartamento T2 Implex em construção A...,440 000 €,"Glória e Vera Cruz, Aveiro, Aveiro",T2,97.78,https://www.imovirtual.com/pt/anuncio/espacoso...,2025-01-21,NaN,97.78m²,Aquecimento:,Aquecimento:,NaN,NaN,NaN,97.78m²,Aquecimento:,Aquecimento:,NaN,NaN
78,78,Espaçoso apartamento T3 Implex em construção A...,620 000 €,"Glória e Vera Cruz, Aveiro, Aveiro",T3,138.18,https://www.imovirtual.com/pt/anuncio/espacoso...,2025-01-21,NaN,138.18m²,Aquecimento:,Aquecimento:,NaN,NaN,NaN,138.18m²,Aquecimento:,Aquecimento:,NaN,NaN


In [3]:
base_url = "https://www.imovirtual.com/comprar/apartamento/?page="
num_pages = 2  # Specify the number of pages to scrape
df = []
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

In [4]:
def scrape_data(url):
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to retrieve data. HTTP Status code: {response.status_code}")
        return None

    soup = BeautifulSoup(response.content, 'html.parser')
    article_elements = soup.find_all('article', {'data-cy': 'listing-item'})

    # Lists to store extracted data
    titles = []
    prices = []
    locations = []
    rooms = []
    areas = []
    urls = []
    dates = []

    scrape_date = datetime.now().strftime('%Y-%m-%d')

    for article_element in article_elements:
        # Extract title
        title_element = article_element.find('p', class_='css-u3orbr e1g5xnx10')
        titles.append(title_element.text.strip() if title_element else np.nan)

        # Extract URL
        url_element = article_element.find('a')
        urls.append(f"https://www.imovirtual.com{url_element['href']}" if url_element else np.nan)

        # Extract price
        price_element = article_element.find('span', class_='css-2bt9f1 evk7nst0')
        prices.append(price_element.text.strip() if price_element else np.nan)

        # Extract location
        location_element = article_element.find('p', class_='css-42r2ms eejmx80')
        locations.append(location_element.text.strip() if location_element else np.nan)

        # Extract room count and area from <dl>
        dl_element = article_element.find('dl', class_='css-12dsp7a e1clni9t1')
        room = np.nan
        area = np.nan

        if dl_element:
            dt_elements = dl_element.find_all('dt')
            dd_elements = dl_element.find_all('dd')

            # Ensure there is a match between <dt> and <dd>
            for dt, dd in zip(dt_elements, dd_elements):
                if dt.text.strip() == "Tipologia":
                    room = dd.text.strip()
                if dt.text.strip() == "Zona":
                    area = dd.text.strip().split(" ")[0]  # Extract just the numeric value
                    break

        rooms.append(room)
        areas.append(area)

        # Add scraping date for each item
        dates.append(scrape_date)
    
    print(f"Titles: {len(titles)}, Prices: {len(prices)}, Locations: {len(locations)}, Rooms: {len(rooms)}, Areas: {len(areas)}, URLs: {len(urls)}")

    return {
        'titles': titles,
        'prices': prices,
        'locations': locations,
        'rooms': rooms,
        'areas': areas,
        'urls': urls,
        'dates': dates
    }

In [5]:
titles = []
prices = []
locations = []
rooms = []
areas = []
urls = []
dates = []

for page in range(1, num_pages + 1):
    print(f"Processing page {page}...")
    page_url = base_url + str(page)
    
    # Scrape data for the current page
    scraped_data = scrape_data(page_url)
    if not scraped_data:
        print(f"Skipping page {page} due to scraping error.")
        continue

    page_titles = scraped_data["titles"]
    page_prices = scraped_data["prices"]
    page_locations = scraped_data["locations"]
    page_rooms = scraped_data["rooms"]
    page_areas = scraped_data["areas"]
    page_urls = scraped_data["urls"]
    page_dates = scraped_data["dates"]

    # Validate that all lists have the same length
    lengths = [
        len(page_titles),
        len(page_prices),
        len(page_locations),
        len(page_rooms),
        len(page_areas),
        len(page_urls),
        len(page_dates)
    ]
    
    if len(set(lengths)) != 1:
        print(f"Warning: Data length mismatch on page {page}. Skipping...")
        print(f"Lengths: {lengths}")
        continue

    # Append to the main lists
    titles += page_titles
    prices += page_prices
    locations += page_locations
    rooms += page_rooms
    areas += page_areas
    urls += page_urls
    dates += page_dates

    print(f"Page {page} processed: {lengths[0]} items.")

Processing page 1...
Titles: 33, Prices: 33, Locations: 33, Rooms: 33, Areas: 33, URLs: 33
Page 1 processed: 33 items.
Processing page 2...
Titles: 33, Prices: 33, Locations: 33, Rooms: 33, Areas: 33, URLs: 33
Page 2 processed: 33 items.


In [6]:
data = {
    "Title": titles,
    "Price": prices,
    "Location": locations,
    "Rooms": rooms,
    "Area": areas,
    "URL": urls,
    "DateScraped": dates
}

df = pd.DataFrame(data)

In [7]:
df.to_csv("Files/apartmentsPortugal.csv", index="0")

In [21]:
from fake_useragent import UserAgent
import time
import random

# Load your DataFrame containing property URLs
df = pd.read_csv('Files/apartmentsPortugal.csv')

# Create empty columns in the existing DataFrame to store the extracted information
df['Useful area'] = None
df['Gross area'] = None
df['Construction year'] = None
df['Energetic certificate'] = None
df['Enterprise'] = None
df['Rooms'] = None
df['Bathroom'] = None
df['Description'] = None

# Initialize UserAgent object
ua = UserAgent()

url_counter = 0

for property_number, url in enumerate(df['URL']):
    # Send a request to the URL with a random User-Agent
    headers = {'User-Agent': ua.random}
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        try:
            # Extract useful_area if available (if not, set to 'N/A')
            useful_area = "N/A"
            df.at[property_number, 'Useful area'] = useful_area
        except AttributeError:
            pass

        try:
            # Find all <button> elements with the target class
            buttons = soup.find_all('button', class_='eezlw8k1 css-ds0a69')
            
            # Ensure there are at least three buttons for area, rooms, and bathrooms
            if len(buttons) >= 3:
                # Extract area from the first button
                area_div = buttons[0].find('div', class_='css-1ftqasz')
                if area_div:
                    area = area_div.get_text(strip=True)
                    df.at[property_number, 'Gross area'] = area
                else:
                    df.at[property_number, 'Gross area'] = "N/A"
                
                # Extract rooms from the second button
                rooms_div = buttons[1].find('div', class_='css-1ftqasz')
                if rooms_div:
                    rooms = rooms_div.get_text(strip=True)
                    df.at[property_number, 'Rooms'] = rooms
                else:
                    df.at[property_number, 'Rooms'] = "N/A"
                
                # Extract bathrooms from the third button
                bathrooms_div = buttons[2].find('div', class_='css-1ftqasz')
                if bathrooms_div:
                    bathrooms = bathrooms_div.get_text(strip=True)
                    df.at[property_number, 'Bathroom'] = bathrooms
                else:
                    df.at[property_number, 'Bathroom'] = "N/A"
            else:
                # If fewer than three buttons are found, set all fields to "N/A"
                df.at[property_number, 'Area'] = "N/A"
                df.at[property_number, 'Rooms'] = "N/A"
                df.at[property_number, 'Bathrooms'] = "N/A"

        except AttributeError:
            # Handle potential errors gracefully
            df.at[property_number, 'Area'] = "N/A"
            df.at[property_number, 'Rooms'] = "N/A"
            df.at[property_number, 'Bathrooms'] = "N/A"
            pass


 
        try:
            # Find all div elements with the target class
            divs = soup.find_all('div', class_='css-t7cajz e15n0fyo1')
            
            construction_year_found = False  # Flag to indicate if the construction year was found
            
            for div in divs:
                # Find all <p> elements inside the current div
                p_elements = div.find_all('p', class_='e15n0fyo2 css-nlohq6')
                
                # Iterate through the <p> elements to find the "Ano de construção"
                for i, p in enumerate(p_elements):
                    if "Ano de construção" in p.get_text(strip=True):
                        # Ensure there is a next <p> element for the construction year
                        if i + 1 < len(p_elements):
                            construction_year = p_elements[i + 1].get_text(strip=True)
                            df.at[property_number, 'Construction year'] = construction_year
                            construction_year_found = True
                            break
                if construction_year_found:
                    break
            
            # If no construction year was found in any of the divs
            if not construction_year_found:
                df.at[property_number, 'Construction year'] = "N/A"

        except (AttributeError, IndexError):
            # Handle potential errors gracefully
            df.at[property_number, 'Construction year'] = "N/A"
            pass


        try:
            # Extract energetic_certificate
            energetic_certificate = soup.find('p', class_='e15n0fyo2 css-nlohq6')
            if energetic_certificate:
                df.at[property_number, 'Energetic certificate'] = energetic_certificate.get_text(strip=True)
            else:
                df.at[property_number, 'Energetic certificate'] = "N/A"
        except AttributeError:
            pass

        try:
            # Extract enterprise if available
            enterprise = "N/A"
            df.at[property_number, 'Enterprise'] = enterprise
        except AttributeError:
            pass

        try:
            # Extract description
            description = soup.find('div', {'data-cy': 'adPageAdDescription'}).text.strip()
            df.at[property_number, 'Description'] = description
        except AttributeError:
            pass

        # Introduce a random delay between requests (e.g., between 0.5 and 1.5 seconds)
        delay = random.uniform(0.5, 1.5)
        time.sleep(delay)
        # Increment the URL counter
        url_counter += 1

        # Check if 100 URLs have been processed and save the DataFrame
        if url_counter % 100 == 0:
            df.to_csv('Files/apartmentsPortugal.csv', index=False)
            print(f"Saved progress at {url_counter} URLs.")

    else:
        print(f"Failed to retrieve property page for {url}")

# Save the updated DataFrame to the existing CSV file
df.to_csv('Files/apartmentsPortugal.csv', index=False)

print("Information extracted and added to apartmentsPortugal.csv.")


Information extracted and added to apartmentsPortugal.csv.


In [22]:
df

,Unnamed: 0,Title,Price,Location,Rooms,Area,URL,DateScraped,Useful area,Gross area,Construction year,Energetic certificate,Enterprise,Description,Bathroom,Bathrooms
0,0,Apartamento T2 com varanda em novo empreendime...,1 465 000 €,"Conde Barão, Misericórdia, Lisboa, Lisboa",T2,146.0,https://www.imovirtual.com/pt/anuncio/apartame...,2025-01-21,N/A,146m²,N/A,Aquecimento:,N/A,,2,NaN
1,1,Apartamento T1 em novo empreendimento na Graça,355 000 €,"Graça, São Vicente, Lisboa, Lisboa",T1,52.0,https://www.imovirtual.com/pt/anuncio/apartame...,2025-01-21,N/A,52m²,N/A,Aquecimento:,N/A,,1,NaN
2,2,Apartamento T1 com varanda em novo empreendime...,388 850 €,"Santo António dos Cavaleiros, Santo António do...",T1,63.0,https://www.imovirtual.com/pt/anuncio/apartame...,2025-01-21,N/A,63m²,N/A,Aquecimento:,N/A,,1,NaN
3,3,Apartamento T1 com varanda NOVO em Matosinhos Sul,260 000 €,"Zona Histórica de Matosinhos, Matosinhos e Leç...",N/A,N/A,https://www.imovirtual.com/pt/anuncio/apartame...,2025-01-21,N/A,None,N/A,Aquecimento:,N/A,,None,N/A
4,4,T3 sala a poente com vista mar em Matosinhos,470 000 €,"Zona Histórica de Matosinhos, Matosinhos e Leç...",T3,127.15,https://www.imovirtual.com/pt/anuncio/t3-sala-...,2025-01-21,N/A,127.15m²,N/A,Aquecimento:,N/A,,3,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,61,"NOVO T2 c/ garagem 24m2 + arrumo de 94m2, a 50...",283 000 €,"Argoncilhe, Santa Maria da Feira, Aveiro",T2,163.0,https://www.imovirtual.com/pt/anuncio/novo-t2-...,2025-01-21,N/A,163m²,2024,Aquecimento:,N/A,,4 ou mais,NaN
62,62,"Apartamento T4, com 3 suites, terraço e garage...",670 000 €,"Prelada - Monte dos Burgos, Ramalde, Porto, Porto",T4,166.0,https://www.imovirtual.com/pt/anuncio/apartame...,2025-01-21,N/A,166m²,N/A,Aquecimento:,N/A,,4 ou mais,NaN
63,63,Prédio Totalmente Remodelado com Negócio de AL...,1 300 000 €,"Rua de Campolide, Nova Campolide, Campolide, L...",N/A,N/A,https://www.imovirtual.com/pt/anuncio/predio-t...,2025-01-21,N/A,None,N/A,Aquecimento:,N/A,,None,N/A
64,64,"Duplex T3 + 1 com 201m2 construídos, 178m2 úte...",349 000 €,"Bairro Santana - Cobre, Cascais e Estoril, Cas...",T4,178.0,https://www.imovirtual.com/pt/anuncio/duplex-t...,2025-01-21,N/A,178m²,N/A,Aquecimento:,N/A,,2,NaN


In [ ]:
# Load your DataFrame containing property URLs
df = pd.read_csv('Files/apartmentsPortugal.csv')

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

for property_number, url in enumerate(df['URL']):
    # Send a request to the URL
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find the script tag containing image data
        script_elements = soup.find_all('script')

        for script_element in script_elements:
            script_content = script_element.string
            if script_content:
                # Use regex to extract image URLs from the script
                image_urls = re.findall(r'"large":"(https://ireland\.apollo\.olxcdn\.com[^"]+)"', script_content)
                if image_urls:
                    property_folder = f'Files/PropertiesImages/Property{property_number}'
                    os.makedirs(property_folder, exist_ok=True)

                    for index, image_url in enumerate(image_urls):
                        image_response = requests.get(image_url)
                        if image_response.status_code == 200:
                            image_extension = "webp"  # Assuming images are in webp format
                            image_filename = f"Property{property_number}-Image{index + 1}.{image_extension}"
                            image_path = os.path.join(property_folder, image_filename)

                            with open(image_path, 'wb') as f:
                                f.write(image_response.content)
                            print(f"Image {index + 1} saved to {image_path}")
                        else:
                            print(f"Failed to download image {index + 1}")
                else:
                    print("No image URLs found in script")
    else:
        print(f"Failed to retrieve property page for {url}")

print("Images downloaded and saved for all properties.")


In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
import re

# Load your DataFrame containing property URLs
df = pd.read_csv('Files/apartmentsPortugal.csv')

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Choose  a property to start from
start_property = 13843  

for property_number, url in enumerate(df['URL']):
    if property_number < start_property:
        continue  # Pula propriedades até atingir a propriedade inicial

    # Send a request to the URL
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find the script tag containing image data
        script_elements = soup.find_all('script')

        for script_element in script_elements:
            script_content = script_element.string
            if script_content:
                # Use regex to extract image URLs from the script
                image_urls = re.findall(r'"large":"(https://ireland\.apollo\.olxcdn\.com[^"]+)"', script_content)
                if image_urls:
                    property_folder = f'Files/PropertiesImages/Property{property_number}'
                    os.makedirs(property_folder, exist_ok=True)

                    for index, image_url in enumerate(image_urls):
                        image_response = requests.get(image_url)
                        if image_response.status_code == 200:
                            image_extension = "webp"  # Assuming images are in webp format
                            image_filename = f"Property{property_number}-Image{index + 1}.{image_extension}"
                            image_path = os.path.join(property_folder, image_filename)

                            with open(image_path, 'wb') as f:
                                f.write(image_response.content)
                            print(f"Image {index + 1} saved to {image_path}")
                        else:
                            print(f"Failed to download image {index + 1}")
                else:
                    print("No image URLs found in script")
    else:
        print(f"Failed to retrieve property page for {url}")

print("Images downloaded and saved for all properties.")
